In [ ]:
''''
The seting of mixed_digit is following FedBN (this paper url={https://openreview.net/pdf?id=6YEQUn0QICG}).
Code souce of data process: https://github.com/med-air/FedBN

Before running the this file, due to the requirement of maximum file size of Supplementary Material,
please download the pre-processed datasets from following url:
    https://drive.google.com/uc?export=download&id=1moBE_ASD5vIOaU8ZHm_Nsj0KAfX5T0Sf
    
and unzip it under 'data/mixed_digit_dataset' directory,
then you can start following experiments on mixed-digit dataset.

Similarly, for pacs, please download the datasets from following url:
https://datasets.activeloop.ai/docs/ml/datasets/pacs-dataset/

and unzip it under 'data/PACS' directory,
then you can start following experiments on PACS dataset.
''''

In [ ]:
import torch
import torch.nn.functional as F

import random, os
import numpy as np
from math import sqrt
from matplotlib import pyplot as plt

from args_mixed_digit_pacs import args_parser
import server_se1 as server
import model

from utils.global_test import test_on_globaldataset_mixed_digit
from utils.training_loss import train_loss_show

args = args_parser()


def seed_torch(seed=args.seed):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

seed_torch()
GLOBAL_SEED = 1
def worker_init_fn(worker_id):
    global GLOBAL_WORKER_ID
    GLOBAL_WORKER_ID = worker_id
    set_seed(GLOBAL_SEED + worker_id)

In [ ]:
similarity = False
training_loss_show =True
init_model = model.Client_Model(args, name=args.model_name).to(args.device)

C = f"{args.split}"
print(C)
D = f"{args.r}r_lr{args.lr}_decay{round(1-args.weight_decay, 4)}_M{args.momentum}_B{args.B}_C{args.C}_fima{args.r_ima}_W{args.window_size}_lrdecay{round(1-args.lr_ima_decay, 4)}_{args.dataset}_{args.K}"
print(D)
args.setup = D

In [ ]:
from mixed_digit_dataloader import *
from PACS_dataloader import  *
 
if args.dataset == 'mixed_digit':
    _, testset = get_mixed_digit_datasets()
    Dtrs = get_mixed_digit_client_dataloaders(args)
elif args.dataset == 'pacs':
    _, testset = get_PACS_datasets(random_seed=args.seed)
    Dtrs = get_PACS_dataloaders(args)

for k in range(args.K):
    if len(Dtrs[k]) ==0:
        print(k)

trainset = []

dict_users_train = [[] for k in range(args.K)]

for k in range(args.K):
    dict_users_train[k] = np.empty(len(Dtrs[k].dataset))

if 'mixed_digit' in args.dataset:
    dict_test = {'MNIST':[], 'SVHN':[], 'USPS':[], 'SynthDigits':[], 'MNIST-M':[]}
    datasets_name = ['MNIST', 'SVHN', 'USPS', 'SynthDigits', 'MNIST-M']
elif 'pacs' in args.dataset:
    dict_test = {'photo':[], 'art_painting':[], 'cartoon':[], 'sketch':[]}
    datasets_name = ['photo', 'art_painting', 'cartoon', 'sketch']

for index, testset_per in enumerate(testset):
    dict_test[datasets_name[index]] = testset_sampling_mixed_digit(args, testset_per, 1000)

In [ ]:
#  baseline---->fedavg
print('baseline---->fedavg')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedavg = server.Server(args, specf_model, trainset, dict_users_train)
global_model1, _, _, loss_dict1, clients_index1, acc_list1 = server_fedavg.fedavg(testset, dict_test, 
                                                                                  agg_mode='ima',
                                                                                 test_global_model_accuracy = True)

In [ ]:
for index, testset_per in enumerate(testset):
    acc1, loss1 = test_on_globaldataset_mixed_digit(args, global_model1, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc1}, loss1:{loss1}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dict1, clients_index1)

In [ ]:
del server_fedavg

In [ ]:
#  baseline---->fedprox 
print('baseline---->fedprox')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedprox = server.Server(args, specf_model, trainset, dict_users_train)
# server_fedprox_joint = copy.deepcopy(serverz)

global_modelp, _, _, loss_dictp, clients_indexp, acc_listp = server_fedprox.fedprox(testset, dict_test,  
                                                                                    agg_mode='ima',
                                                                                    test_global_model_accuracy = True)


In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelp, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictp,clients_indexp)

In [ ]:
del server_fedprox

In [ ]:
#  baseline---->fedasam
print('baseline---->fedasam')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedasam = server.Server(args, specf_model, trainset, dict_users_train)
# server_fedprox_joint = copy.deepcopy(serverz)

global_modelasam, _, _, loss_dictasam, clients_indexasam, acc_listasam = server_fedasam.fedsam(testset, dict_test,  
                                                                                    agg_mode='ima',
                                                                                    test_global_model_accuracy = True)


In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelasam, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictasam,clients_indexasam)

In [ ]:
del server_fedasam

In [ ]:
#  baseline---->fednova
print('baseline---->fednova')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fednova = server.Server(args, specf_model, trainset, dict_users_train)

global_modelnova, _, _, loss_dictnova, clients_indexnova, acc_listnova = server_fednova.fednova(testset, dict_test,  
                                                                                    agg_mode='fednova+ima',
                                                                                    test_global_model_accuracy = True)


In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelnova, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictnova,clients_indexnova)

In [ ]:
del server_fednova

In [ ]:
#  baseline---->fedadam
print('baseline---->fedadam')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedadam = server.Server(args, specf_model, trainset, dict_users_train)

global_modeladam, _, _, loss_dictadam, clients_indexadam, acc_listadam = server_fedadam.fedavg(testset, dict_test,  
                                                                                    agg_mode='fedadam+ima',
                                                                                    test_global_model_accuracy = True)


In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modeladam, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictadam,clients_indexadam)

In [ ]:
del server_fedadam

In [ ]:
#  baseline---->fedyogi
print('baseline---->fedyogi')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedyogi = server.Server(args, specf_model, trainset, dict_users_train)

global_modelyogi, _, _, loss_dictyogi, clients_indexyogi, acc_listyogi = server_fedyogi.fedavg(testset, dict_test,  
                                                                                    agg_mode='fedyogi+ima',
                                                                                    test_global_model_accuracy = True)


In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelyogi, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictyogi,clients_indexyogi)

In [ ]:
del server_fedyogi

In [ ]:
#  baseline---->fedgma
print('baseline---->fedgma')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_fedgma = server.Server(args, specf_model, trainset, dict_users_train)

global_modelgma, _, _, loss_dictgma, clients_indexgma, acc_listgma = server_fedgma.fedavg(testset, dict_test,
                                                                                          agg_mode='gma+ima',
                                                                                          test_global_model_accuracy = True)

                                                                                                                               

In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelgma, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictgma, clients_indexgma)

In [ ]:
del server_fedgma

In [ ]:
#fedfa
print('baseline---->fedfa')

In [ ]:
specf_model = copy.deepcopy(init_model)
server_feature = server.Server(args, specf_model, trainset, dict_users_train)
# server_feature = copy.deepcopy(serverz)

global_modelfa, _, _, loss_dictfa, clients_indexfa, acc_listfa = server_feature.fedfa_anchorloss(testset, 
                                                                                                 dict_test,
                                                                                                 agg_mode='ima',
                                                                                                 test_global_model_accuracy = True)



In [ ]:
for index, testset_per in enumerate(testset):
    acc, loss = test_on_globaldataset_mixed_digit(args, global_modelfa, testset_per, 
                                                dict_test[datasets_name[index]])
    print(f"{datasets_name[index]}：acc1:{acc}, loss1:{loss}")

In [ ]:
if training_loss_show:
    train_loss_show(args, loss_dictfa,clients_indexfa)

In [ ]:
del server_feature

In [ ]:
plt.figure(figsize=(25,3))
for i,j in enumerate(datasets_name):
    plt.subplot(1,5,i+1)
    plt.plot(acc_list1[j],label='FedAvg+IMA')
    plt.plot(acc_listp[j],label='FedProx+IMA')
    plt.plot(acc_listasam[j],label='FedASAM+IMA')
    plt.plot(acc_listnova[j],label='FedNova+IMA')
    plt.plot(acc_listadam[j],label='FedAdam+IMA')
    plt.plot(acc_listyogi[j],label='FedYogi+IMA')
    plt.plot(acc_listgma[j],label='FedGMA+IMA')
    plt.plot(acc_listfa[j],label='FedFA+IMA')
    plt.title(j)
    plt.xlabel('Rounds')
    plt.ylabel('Acc.')
plt.legend()
# plt.savefig(f'acc_comparison_allmethods_{args.model_name}_{args.dataset}_ima_{args.split}.pdf')